# An example to generate STM from dynamic estimation

adapted from https://github.com/TUDelftGeodesy/DePSI_group/blob/dev/examples/notebooks/demo_dynamic_estimation.ipynb

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# from depsi.LAMBDA import *
from depsi.dynamic_estimation import lambda_estimation

# Set input parameters

In [ ]:
# set wavelength of Sentinel-1 (C-band)
wavelength = 0.055465763 # m
init_len = 50
sigma_acc = 0.003 # m/yr^2
L = 30/365

# Initial gauss for the sigma of the unknown parameters
sigma_offset = 0.001 #m
sigma_vel =  0.0001 #m/yr
sigma_h = 5 #m
sigma_ther = 0.00005 #m/°C

# the option for the LAMBDA method
# method == 1: ILS with shrinking search
# method == 2: Integer rounding
# method == 3: Integer bootstrapping
# method == 4: PAR
# method == 5: ILS with Ratio Test
method = 3

# Load the STM derived from PS selection

In [ ]:
# Load all to memory
stm = xr.open_zarr('../../data/stm_amsterdam_173p.zarr').compute()
stm

# Choose the reference point

In [ ]:
# Select the point with the smallest NMAD for initialization as the reference point
nmad_init = np.array(stm['nmad_init'])
ref_pnt_idx= int(np.argmin(nmad_init))
ref_pnt_idx

# Define functions

In [ ]:
def polynomial_fitting(x, y, degree=1):
    """
    Perform polynomial fitting on the input data (x, y).

    Parameters:
    
    x : array_like
        Independent variable data points.
    y : array_like 
        Dependent variable data points.
    degree: int
        Degree of the polynomial fit (default is 1 for linear).

    Returns:
    y_fitted : ndarray
        Fitted y values for the given x based on the polynomial fit.
    poly_fn : np.poly1d
        Polynomial function that represents the fit.
    coeffs : ndarray 
        Coefficients of the fitted polynomial.
    """
    # Validate inputs
    if len(x) != len(y):
        raise ValueError("Input arrays 'x' and 'y' must have the same length.")
    if degree < 1:
        raise ValueError("Degree must be at least 1.")

    # Perform polynomial fitting
    coeffs = np.polyfit(x, y, degree)
    poly_fn = np.poly1d(coeffs)
    y_fitted = poly_fn(x)

    return y_fitted, poly_fn, coeffs



def NMAD_to_sigma_phase(nmad, method):

    """
    Converts Normalized Median Absolute Deviation (NMAD) to the sigma of phase observations.

    This function uses an empirical cubic approximation (and is not based on physics):
    sigma = a + b*nmad + c*nmad**2 + d*nmad**3

    Parameters:
    nmad : array_like
        Normalized Median Absolute Deviation value.
    method : {'mean', 'mean_2_sigma'}
        Method used to compute sigma.
    
    Returns:
    sigma: ndarray
        Estimated sigma value.
    """
    if method == 'mean':
        a, b, c, d = -0.0144869469, 2.00028682, -5.23271341, 21.1111801
    elif method == 'mean_2_sigma':
        a, b, c, d = 0.01907808, 1.2852969, 1.90052824, 11.60677721
    else:
        raise ValueError("Invalid method. Choose 'mean' or 'mean_2_sigma'.")

    sigma = a + b*nmad + c*nmad**2 + d*nmad**3
    
    return sigma

## Perform unwrapping with lambda estimation

In [ ]:
# select the reference point
stm_refpnt = stm.isel(space=ref_pnt_idx)

# subset the data for debug
NUM_POINTS = 5
stm = stm.isel(space=range(5))

In [ ]:
# Initiate empty arrays to store results
x_hat = np.zeros((stm.sizes["space"], 4))
Q_xhat = np.zeros(( 4, 4, stm.sizes["space"]))
y_hats = np.zeros((stm.sizes["space"], stm.sizes["time"]))
phs_unw_init = np.zeros((stm.sizes["space"], stm.sizes["time"]))


# Loop over all points and perform phase unwrapping
for pnt_id in range(stm.sizes["space"]):
    # Select current point
    stm_1pnt = stm.isel(space=pnt_id)

    print(f"{pnt_id}/{stm.sizes['space']}")

    # Get the sigma for the arc with the NMAD from the incremental time series
    sigma_nmad_inc_i = NMAD_to_sigma_phase(
        stm_1pnt["nmad_inc_stm"].data, "mean_2_sigma"
    )
    sigma_nmad_inc_j = NMAD_to_sigma_phase(
        stm_refpnt["nmad_inc_stm"].data, "mean_2_sigma"
    )
    sigma_nmad_inc_arc = np.sqrt(
        np.square(sigma_nmad_inc_i) + np.square(sigma_nmad_inc_j)
    )

    # Compute arc phase
    phs_wrapped_arc = np.angle(
        stm_refpnt["sd_complex"].data * stm_1pnt["sd_complex"].data.conj()
    )

    # Compute 'mean' h2ph value for the arc (which we currently model as the average of the two time series)
    h2ph_arc = (stm_refpnt["h2ph_values"].data + stm_1pnt["h2ph_values"].data) / 2

    results = lambda_estimation(
        wavelength=wavelength,
        phs_wrapped=phs_wrapped_arc,
        sigma_phs_apri=sigma_nmad_inc_arc,
        years=stm_1pnt["years"].data,
        h2ph_arc=h2ph_arc,
        temp=stm_1pnt["temperature"].data,
        sigma_offset=sigma_offset,
        sigma_vel=sigma_vel,
        sigma_h=sigma_h,
        sigma_ther=sigma_ther,
        method=method,
    )

    # Store results
    x_hat[pnt_id, :] = results[0]
    Q_xhat[:, :, pnt_id] = results[1]
    y_hats[pnt_id, :] = results[2]
    phs_unw_init[pnt_id, :] = results[3]

In [ ]:
# For now, write x_hat and Q_xhat to npz files, since they so not fit in dimensions of an STM
np.savez(
    "./x_hat_Q_xhat.npz",
    x_hat=x_hat,
    Q_xhat=Q_xhat,
)

In [ ]:
# Attach y_hats and phs_unw_init to the STM
stm["y_hats"] = (("space", "time"), y_hats)
stm["phs_unw_init"] = (("space", "time"), phs_unw_init)
stm

In [ ]:
stm.to_zarr(
    "./stm_amsterdam_173p_init_unw.zarr",
    mode="w",
)